<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

We will convert addresses into their equivalent latitude and longitude values. Also, we will use the Foursquare API to explore neighborhoods in Toronto. We will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. We will use the *k*-means clustering algorithm to complete this task. Finally, we will use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Setup all the python libraries dependencies that we will need.

In [2]:
import numpy as np    # library to handle data in a vectorized manner

import pandas as pd                     # library for data analsysis
pd.set_option ('display.max_columns', None)
pd.set_option ('display.max_rows', None)

import json                             # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim   # convert an address into latitude and longitude values

import requests                           # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium                    # map rendering library

# !pip install beautifulsoup4
from bs4 import BeautifulSoup    # pull in HTML screen scraper support

import csv

print ('python Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

<a id='item1'></a>

## 1. Download and Explore Dataset

In order to segement the neighborhoods and explore them, we will need a dataset that contains the boroughs and the neighborhoods that exist within each borough, as well as the the latitude and logitude coordinates of each neighborhood. 

This dataset exists for free on the web. Here is the link to the dataset: 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

So we need to download this HTML dataset, and then use the BeautifulSoup screen scraper to extract the data we need.

In [3]:
url = requests.get ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup (url.text, 'lxml')

#print (soup.prettify())

print ('Toronto HTML Table data downloaded!')

Toronto HTML Table data downloaded!


#### Load and explore the data

Next, let's load the data.

In [4]:
# Generate a CSV file from the Web screen scraped data that we parsed from BeautifulSoup
#with open ('toronto_postal.csv', 'w', newline='') as csv_file:
csv_output_file = open ('toronto_postal.csv', 'w')
csv_writer = csv.writer (csv_output_file)
csv_writer.writerow (['Postcode', 'Borough', 'Neighbourhood']) # write out header line

# An examination of soup.prettify output shows that the data to be scraped
# is laid out as an HTML table, using:
# <tr> (row begin) <td>xxx</td> <td>yyy</td> (detail columns) </tr> (row end)
# e.g. one of the table entries is shown below:
# <tr> <td>M6A</td> <td><a href="/wiki/North_York" title="North York">North York</a></td>
# <td><a href="/wiki/Lawrence_Manor" title="Lawrence Manor">Lawrence Manor</a></td> </tr>

# Scrape the data from the HTML table, using the <tr> and <td> tags
for tr_row in soup.find_all('tr')[1:]:
     td_col = tr_row.find_all ('td')
     if len(td_col) == 3:
        Postcode = td_col[0].text
        Borough  = td_col[1].text
        Neighbourhood = td_col[2].text
     #print (Postcode, Borough, Neighbourhood)
     csv_writer.writerow ([Postcode, Borough, Neighbourhood])
        
csv_output_file.close()

In [6]:
# convert the CSV file into a pandas dataframe, then print the first few entries

df_toronto_raw = pd.read_csv ('toronto_postal.csv')

df_toronto_raw.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [7]:
# strip off the trailing newline \n and carriage return \r crap left over from the screen scraping CSV conversion

df_toronto_raw [df_toronto_raw.columns] = df_toronto_raw.apply (lambda s: s.str.strip('\n'))
df_toronto_raw [df_toronto_raw.columns] = df_toronto_raw.apply (lambda s: s.str.strip('\r'))

print ('Raw Torornto Neighborhoods shape is: ', df_toronto_raw.shape)
df_toronto_raw.head (10)

Raw Torornto Neighborhoods shape is:  (294, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


## Data Wrangling/Cleaning

As per class instructions, clean up the data we recieved.

In [8]:
# Per class instructions, ignore (strip out) any cells with Borough = Not Assigned

df_clean1 = df_toronto_raw[df_toronto_raw.Borough != 'Not assigned'] # copy all entries except those set to Not A

df_clean1.sort_values (by=['Postcode'],ascending=True,inplace=True)  # sort into order and reset index to 0,1,2,
df_clean1 = df_clean1.reset_index (drop=True)

df_clean1.head (15)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek
5,M1E,Scarborough,Guildwood
6,M1E,Scarborough,Morningside
7,M1E,Scarborough,West Hill
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


In [9]:
# Per class instructions, merge any entries where the multiple neighborhoods
# exist in same postal code e.g. M5A postal code is listed twice because
# it contains two bneighborhoods: Harbourfront and Regent Park.
# We merge these using a Pandas groupby clause with an join option.
# Join them with a comma between the merged entities.

df_clean1.set_index (['Postcode','Borough'],inplace=True)
df_merge_hood = df_clean1.groupby (level=['Postcode','Borough'], sort=False).agg( ','.join)
df_clean1.reset_index()         # release index
#df_merge_hood.reset_index()    # clear any remnant set iundex flags

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek
5,M1E,Scarborough,Guildwood
6,M1E,Scarborough,Morningside
7,M1E,Scarborough,West Hill
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


In [10]:
df_merge_hood.head (20)

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"


In [15]:
# Per the class instructions, if a Borough has a Not Assigned neightborhood,
# then set the neighboor name to match the Borough,
# e.g. M7A Queen's park has a Not Assigned neighborhood

df_merge_hood.loc[df_merge_hood['Neighbourhood'] =='Not assigned'] = df_merge_hood[df_merge_hood['Neighbourhood'] == 'Not assigned'].index.values[0][1]

df_merge_hood.head (10)         # check results

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"


In [16]:
# Per class instruction, show the shape of the cleaned up dataframe

df_toronto = df_merge_hood.reset_index() # reset index, to normalize all levels back to 1
df_toronto.shape

(103, 3)

In [17]:
# import the CSV file Geospatial_Coordinates.csv referenced by the instructor

df_geospatial = pd.read_csv ('Geospatial_Coordinates.csv')

df_geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
# now merge the geospatial data with the toronto postal data, using
# Postal code as common key. This is equivalent to a SQL Join
# column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

df_toronto_data = pd.merge (df_toronto, df_geospatial, left_on='Postcode', right_on='Postal Code')

df_toronto_data.head (10)         # and check the results

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",M1N,43.692657,-79.264848


In [19]:
print ("Total PostCodes: " + str(df_toronto_data['Postcode'].nunique()))
print ("Total Neighbourhoods: " + str(df_toronto_data['Neighbourhood'].nunique()))
print ("Total Boroughs: " + str(df_toronto_data['Borough'].nunique()))
df_toronto_data.shape

Total PostCodes: 103
Total Neighbourhoods: 103
Total Boroughs: 11


(103, 6)

In [20]:
# Now get the Longitude and Latitude location of Toronto

address    = 'Toronto, CANADA'
geolocator = Nominatim()
location   = geolocator.geocode (address)

latitude   = location.latitude
longitude  = location.longitude

print ('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [22]:
# create map of Toronto using latitude and longitude values from above

map_toronto = folium.Map (location=[latitude, longitude], zoom_start=11)
folium.TileLayer('openstreetmap').add_to (map_toronto)
# add markers to map
for lat, lng, label in zip(df_toronto_data['Latitude'], df_toronto_data['Longitude'], df_toronto_data['Neighbourhood']):
    label = folium.Popup (label, parse_html=True)
    folium.CircleMarker (
                  [lat, lng],
                  radius=5,
                  popup=label,
                  color='blue',
                  fill=True,
                  fill_color='#3186cc',
                  fill_opacity=0.7,
                  parse_html=False).add_to(map_toronto)
                                                                                                      
map_toronto

================================================================
## Setup Four Square credentials
================================================================

In [23]:
# use the 4-Square segmenting/clustering thing
CLIENT_ID = '50BIQWK4SFVUM420YXDPJCP2BGLEJORINZQAZF0SHQEREVIX' # my Foursquare ID
CLIENT_SECRET = '4OF5PFQ0PF4JX5RVA3FF52TNZMBU2JZRIQ00JCH1UDOHHKCW' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version or '20180604'
# CAUTION LIMIT must be same as shape (103). If set to 100 (default) later
# processing dies because the frame shapes don't match (103 vs 100)
LIMIT = str(df_toronto_data.shape[0])
print ('Your credentials:')
print ('CLIENT_ID: ' + CLIENT_ID)
print ('CLIENT_SECRET: ' + CLIENT_SECRET)
print ('LIMIT: ' + LIMIT)

Your credentials:
CLIENT_ID: 50BIQWK4SFVUM420YXDPJCP2BGLEJORINZQAZF0SHQEREVIX
CLIENT_SECRET: 4OF5PFQ0PF4JX5RVA3FF52TNZMBU2JZRIQ00JCH1UDOHHKCW
LIMIT: 103


=========================================
## Explore a Single Neighborhood
=========================================

First, do an analysis of just a single neighborhood. In the next step, we will expand this to all the neighborhoods.

In [26]:
first_neighborhood = df_toronto_data.loc[0,'Neighbourhood'] # grab neightborhood in the first row (0) of the Dat

hood_latitude = df_toronto_data.loc[0,'Latitude'] # and get its associated Latitude and longitude loca
hood_longitude = df_toronto_data.loc[0,'Longitude']

print ('Analying Neighborhood {} at Latitude {} and Longitude {}'.format(first_neighborhood, hood_latitude, hood_longitude)) 

Analying Neighborhood Rouge,Malvern at Latitude 43.806686299999996 and Longitude -79.19435340000001


In [27]:
radius = 1000 # define radius of the area to search
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
            
# make the GET request to 4-Square, and get the returned results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c252af74434b91ef296b815'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.715748309000006,
    'lng': -79.58162753348529},
   'sw': {'lat': 43.69774829099998, 'lng': -79.60648126651472}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4be452122457a593c40eaa15',
       'name': 'Tim Hortons',
       'location': {'address': '116 Queens Plate Dr',
        'lat': 43.71465748224628,
        'lng': -79.59371566772467,
        'labeledLatLngs': [{'label': 'display',
        

From the Foursquare lab in Week 2, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [29]:
# function that extracts the category of the venue
def get_category_type (row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [30]:
# read in, and clean, the received JSON data, and put it in a DataFrame

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize (venues)     # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues    = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tim Hortons,Coffee Shop,43.714657,-79.593716
1,Woodbine Hotel & Suites,Hotel,43.712624,-79.592664


And how many venues were returned by Foursquare?

In [31]:
print ('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore All the Neighborhoods in Toronto

#### Create a function to repeat the same process to all the neighborhoods in Toronto

In [33]:
def  getNearbyVenues (names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print (name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append ([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame ([item for venue_list in venues_list for item in venue_list])
    
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return (nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [34]:
# invoke the 4-Square functions for each of the neighborhoods,
# and save the results in a data frame

toronto_venues = getNearbyVenues (names = df_toronto_data['Neighbourhood'],
                                  latitudes = df_toronto_data['Latitude'],
                                  longitudes = df_toronto_data['Longitude']
                                 )

Rouge,Malvern
Port Union,Rouge Hill,Highland Creek
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Golden Mile,Oakridge,Clairlea
Cliffcrest,Scarborough Village West,Cliffside
Cliffside West,Birch Cliff
Wexford Heights,Dorset Park,Scarborough Town Centre
Maryvale,Wexford
Agincourt
Sullivan,Clarks Corners,Tam O'Shanter
Milliken,Agincourt North,L'Amoreaux East,Steeles East
Steeles West,L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Oriole,Henry Farm
Bayview Village
Silver Hills,York Mills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South,Flemingdon Park
Wilson Heights,Downsview North,Bathurst Manor
York University,Northwood Park
Downsview East,CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill,Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beac

#### Check the size and shape of the resulting dataframe

In [35]:
print ("Total Unique Neighbourhoods returned with Venues: " + str(toronto_venues['Neighborhood'].nunique()))
print ('Total number and shape of venues results returned=', toronto_venues.shape)
toronto_venues.head (10)

Total Unique Neighbourhoods returned with Venues: 101
Total number and shape of venues results returned= (2233, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764042,-79.193371,Rental Car Location
7,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
9,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop


### Check how many venues were returned for each neighborhood

In [36]:
toronto_venues.groupby ('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,Richmond,King",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
"Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,Thistletown,Albion Gardens,South Steeles",11,11,11,11,11,11
"Bedford Park,Lawrence Manor East",26,26,26,26,26,26
Berczy Park,55,55,55,55,55,55
Business reply mail Processing Centre969 Eastern,16,16,16,16,16,16
"Cabbagetown,St. James Town",46,46,46,46,46,46
Caledonia-Fairbanks,6,6,6,6,6,6


#### Let's find out how many unique categories can be curated from all the returned venues

In [37]:
print ('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 279 unique categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [38]:
#######################
#  one hot encoding
#######################
toronto_onehot = pd.get_dummies (toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns  = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [39]:
toronto_onehot.shape

(2233, 279)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,Richmond

#### Confirm the new size

In [42]:
toronto_grouped.shape

(101, 279)

#### Print each neighborhood along with the top 5 most common venues

In [45]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print ("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round ({'freq': 2})
    print (temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print ('\n')

----Adelaide,Richmond,King----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2               Skating Rink  0.25
3             Clothing Store  0.25
4  Middle Eastern Restaurant  0.00


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3   Chinese Restaurant  0.25
4               Museum  0.00


----Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,Thistletown,Albion Gardens,South Steeles----
                 venue  freq
0        Grocery Store  0.18
1          Pizza Place  0.09
2          Coffee Shop  0.09
3       Sandwich Place  0.09
4  Japanese Restaurant  0.09


----Bedford Park,Lawrence Manor East----
                  venue  freq
0    

#### Put that venues data into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [46]:
def return_most_common_venues (row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create the new dataframe and display the top 10 venues for each neighborhood.

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append ('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append ('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame (columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,Richmond,King",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Asian Restaurant,Gym,Bakery,Restaurant,Bar
1,Agincourt,Breakfast Spot,Lounge,Clothing Store,Skating Rink,Women's Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,Bayview Village,Bank,Chinese Restaurant,Café,Japanese Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
3,"Beaumond Heights,Humbergate,Jamestown,Mount Ol...",Grocery Store,Fast Food Restaurant,Coffee Shop,Discount Store,Sandwich Place,Japanese Restaurant,Fried Chicken Joint,Beer Store,Pizza Place,Pharmacy
4,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Juice Bar,Fast Food Restaurant,Pharmacy,Restaurant,Butcher,Pub,Café,Pizza Place
5,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Pub,Seafood Restaurant,Farmers Market,Café,Italian Restaurant,Bakery,Cheese Shop
6,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Garden,Comic Shop,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop
7,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Pub,Bakery,Italian Restaurant,Café,Indian Restaurant,Market,Pizza Place,Butcher
8,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Gourmet Shop,Grocery Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant
9,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Gym / Fitness Center,American Restaurant,Fried Chicken Joint,Burrito Place,Mediterranean Restaurant,Sandwich Place,Middle Eastern Restaurant,Dog Run


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [51]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans (n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [52]:
toronto_grouped_clustering.shape

(101, 278)

In [53]:
df_toronto_data.shape

(103, 6)

In [55]:
# Do an inner join, to eliminate any df_toronto_data neighborhoods that did not get results on the 4-square query
# Otherwise, we get blown out in the next step because the shapes don't match: e.g. 103 rows vs 101 rows

df_toronto_data_join = df_toronto_data.rename(columns={'Neighbourhood':'Neighborhood'}) # ensure correct spelling

df_joined_result = toronto_grouped.join (df_toronto_data_join.set_index('Neighborhood'), on='Neighborhood')

df_joined_result.head (10)

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Postcode,Borough,Pos

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [56]:
toronto_merged = df_joined_result

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

################################################################
# merge toronto_grouped with toronto_data PANDAs to add latitude/longitude for each neighborhood
################################################################
toronto_merged = toronto_merged.join (neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Postcode,Borough,Pos

Finally, let's visualize the resulting clusters

In [57]:
# create map
map_clusters = folium.Map (location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange (kclusters)

ys = [i+x+(i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup (str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker (
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Burger Joint,Seafood Restaurant,Bank,Liquor Store,Clothing Store
13,Lincoln Square,Gym / Fitness Center,Theater,Italian Restaurant,Plaza,Concert Hall,French Restaurant,Café,Performing Arts Venue,Indie Movie Theater,Opera House
15,Midtown,Hotel,Food Truck,Steakhouse,Coffee Shop,Theater,Park,Bookstore,Clothing Store,Cocktail Bar,Spa
37,Stuyvesant Town,Bar,Park,German Restaurant,Farmers Market,Pet Service,Harbor / Marina,Cocktail Bar,Heliport,Coffee Shop,Basketball Court


#### Cluster 2

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Discount Store,Supplement Shop,Tennis Stadium,Seafood Restaurant,Gym,Bank,Donut Shop,Sandwich Place,Diner
1,Chinatown,Chinese Restaurant,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Salon / Barbershop,Bakery,Hotpot Restaurant,Ice Cream Shop,Noodle House
3,Inwood,Mexican Restaurant,Lounge,Pizza Place,Café,Pharmacy,Park,Restaurant,Bakery,Frozen Yogurt Shop,Deli / Bodega
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Coffee Shop,Bakery,French Restaurant,Gym / Fitness Center,Cocktail Bar,Hotel,Boutique
9,Yorkville,Coffee Shop,Bar,Gym,Italian Restaurant,Pizza Place,Sushi Restaurant,Deli / Bodega,Wine Shop,Mexican Restaurant,Ice Cream Shop
11,Roosevelt Island,Sandwich Place,Deli / Bodega,Coffee Shop,Rental Car Location,Park,Gym,Liquor Store,School,Baseball Field,Outdoors & Recreation
16,Murray Hill,Hotel,Bar,Japanese Restaurant,Salon / Barbershop,Coffee Shop,Gym,Sandwich Place,Italian Restaurant,American Restaurant,Spa
19,East Village,Bar,Ice Cream Shop,Wine Bar,Cocktail Bar,Mexican Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Speakeasy,Pizza Place,Chinese Restaurant
20,Lower East Side,Café,Chinese Restaurant,Coffee Shop,Ramen Restaurant,Shoe Store,Japanese Restaurant,Sandwich Place,Cocktail Bar,Bakery,Art Gallery
21,Tribeca,Italian Restaurant,American Restaurant,Park,Café,Spa,Wine Bar,Coffee Shop,Boutique,Wine Shop,Gym


#### Cluster 3

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Pizza Place,Bakery,Sandwich Place,Mobile Phone Shop,Mexican Restaurant,Restaurant,Donut Shop,Tapas Restaurant,Supermarket
4,Hamilton Heights,Mexican Restaurant,Café,Coffee Shop,Deli / Bodega,Pizza Place,Caribbean Restaurant,Chinese Restaurant,Bakery,School,Sandwich Place
5,Manhattanville,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Curry Restaurant,Diner,Sushi Restaurant,Supermarket,Spanish Restaurant,Bus Station
6,Central Harlem,African Restaurant,Gym / Fitness Center,French Restaurant,Cosmetics Shop,Chinese Restaurant,Fried Chicken Joint,Seafood Restaurant,American Restaurant,Southern / Soul Food Restaurant,Ethiopian Restaurant
10,Lenox Hill,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Gym / Fitness Center,Deli / Bodega,Sporting Goods Shop,Gym,Burger Joint,Steakhouse
12,Upper West Side,Italian Restaurant,Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Wine Bar,Bakery,Indian Restaurant,Gym / Fitness Center,Thai Restaurant,Seafood Restaurant
14,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,American Restaurant,Coffee Shop,Hotel,Spa,Wine Shop,Gym,Mediterranean Restaurant
17,Chelsea,Italian Restaurant,Coffee Shop,Ice Cream Shop,Bakery,Nightclub,Seafood Restaurant,American Restaurant,Hotel,Theater,Art Gallery
18,Greenwich Village,Italian Restaurant,French Restaurant,Sushi Restaurant,Clothing Store,Electronics Store,Indian Restaurant,Chinese Restaurant,Boutique,Seafood Restaurant,Café
23,Soho,Clothing Store,Boutique,Women's Store,Men's Store,Shoe Store,Art Gallery,Furniture / Home Store,Mediterranean Restaurant,Italian Restaurant,Hotel


#### Cluster 4

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Coffee Shop,Pizza Place,Cosmetics Shop,Café,Yoga Studio,French Restaurant,Spa,Bar,Gym,Japanese Restaurant


#### Cluster 5

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Battery Park City,Coffee Shop,Park,Hotel,Italian Restaurant,Wine Shop,Women's Store,Market,Burger Joint,Fountain,Memorial Site


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).